# Parallel execution of EasyVVUQ workflows with QCG-PilotJob - EasyVVUQ-QCGPJ API
In this Notebook we present how to employ the *EasyVVUQ-QCGPJ* API to enable efficient execution of *EasyVVUQ* workflows on large computing resources. The API extends capabilities of a regular *EasyVVUQ* with the dedicated procedures for interracting with the *QCG-PilotJob* system, enabling the execution of the most demanding parts of the workflow in parallel.

## Installing EasyVVUQ-QCGPJ
The first thing that you need to do in order to install *EasyVVUQ-QCGPJ*, is the installation of its dependencies, namely *EasyVVUQ* and *QCG-PilotJob*. It can be done in the following way:

In [ ]:
pip install easyvvuq qcg-pilotjob

The actual installation of *EasyVVUQ-QCGPJ* is also strightforward:

In [ ]:
pip install easyvvuq-qcgpj

## Getting tutorial files
The next action is to get files that will be used in the tutorial:

In [ ]:
!git clone https://github.com/vecma-project/EasyVVUQ-QCGPJ.git

$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
## A Cooling Coffee Cup with Polynomial Chaos Expansion using EasyVVUQ-QCGPJ

In order to keep the conformance with the basic *EasyVVUQ* tutorial we will show the same workflow as it was used in that tutorial's use-case, i.e. the scenario of Polynomial Chaos Expansion for a model of a cooling coffee cup.  

### The Cooling Coffee Cup model

In order to recall the idea of the scenario, let us shortly describe the physical basics. 

The model uses Newton's law of cooling to
evolve the temperature, $T$, over time ($t$) in an environment at
$T_{env}$ :

$$\frac{dT(t)}{dt} = -\kappa (T(t) -T_{env})$$

The constant $\kappa$ characterizes the rate at which the coffee cup
transfers heat to the environment. In this example we will analyze this
model using the polynomial chaos expansion (PCE) UQ algorithm. We will
use a constant initial temperature $T_0 = 95 ^\circ\text{C}$, and vary
$\kappa$ and $T_{env}$ using a uniform distribution in the ranges
$0.025-0.075$ and $15-25$ respectively.

### EasyVVUQ Script Overview

As it was already said, the origin of the scenario for this tutorial is the UQ analysis of cooling cofee cup model delivered as a primary *EasyVVUQ* example. 
Once the *EasyVVUQ-QCGPJ* sources are checked-out from GitHub, the python code for the model is available in `EasyVVUQ-QCGPJ/tests/cooling/cooling_model.py` and the template used for the encoding of input for this model is available in `EasyVVUQ-QCGPJ/tests/cooling/cooling.template`. 

Since the approach we took to integrate *EasyVVUQ* with *QCG Pilot Job Manager* is considerably non-intrusive, the original *EasyVVUQ* workflow for the scenario (the one without *QCG-PilotJob*) requires only slight modifications to run with the *QCG-PilotJob* mechanism. The crucial ammendment introduced into the current tutorial, is the replacement of basic way of execution of *EasyVVUQ* tasks by *EasyVVUQ-QCGPJ* *encoding* and *execution* routines to perform most demanding computations using the *QCG-PilotJob* system. 
In the rest of this notebook the whole ammended workflow is presented, however in order to not disturb your attention from the main topic of this tutorial, we concentrate on the part related to *EasyVVUQ-QCGPJ* code itself. If you want to know something more about other parts of the workflow please refer to the *EasyVVUQ* documentation or to the basic *EasyVVUQ* tutorial. 

### Setup
In the first step we need to import requied dependencies. In addition to chaospy and easyvvuq we need to import a few classes from *EasyVVUQ-QCGPJ* (`easypj` package)   

In [ ]:
import os
import chaospy as cp
import easyvvuq as uq

from eqi import Executor, Task, TaskType, TaskRequirements, SubmitOrder

Next we will set a couple of helper variables, which are specific for our example:

### Campaign initialisation
EasyVVUQ-QCGPJ doesn't alter the way how EasyVVUQ campaign is created and inited.
The whole initialisation code from the original example can be simply reused with only a minor changes to adjust paths to a local environment. After these small modifications it looks as follows: 

In [ ]:
# Initialize Campaign
my_campaign = uq.Campaign(name='coffee_pce')


params = {
    "temp_init": {"type": "float", "min": 0.0, "max": 100.0, "default": 95.0},
    "kappa": {"type": "float", "min": 0.0, "max": 0.1, "default": 0.025},
    "t_env": {"type": "float", "min": 0.0, "max": 40.0, "default": 15.0},
    "out_file": {"type": "string", "default": "output.csv"}
}

encoder = uq.encoders.GenericEncoder(
    template_fname=os.path.abspath('EasyVVUQ-QCGPJ/tests/cooling/cooling.template'),
    delimiter='$',
    target_filename="cooling_in.json")

decoder = uq.decoders.SimpleCSV(target_filename="output.csv",
                                output_columns=["te"],
                                header=0)

collater = uq.collate.AggregateSamples(average=False)

# Add the app (automatically set as current app)
my_campaign.add_app(name="cooling",
                    params=params,
                    encoder=encoder,
                    decoder=decoder,
                    collater=collater)

# Setup sampler
vary = {
    "kappa": cp.Uniform(0.025, 0.075),
    "t_env": cp.Uniform(15, 25)
}

my_sampler = uq.sampling.PCESampler(vary=vary, polynomial_order=3)
my_campaign.set_sampler(my_sampler)

# Draw samples
my_campaign.draw_samples()

print("Campaign prepared")
print(my_campaign)

### EASYVVUQ-QCGPJ part
In this place we start definitions specific for *EasyVVUQ-QCGPJ*. 
#### EasyVVUQ-QCGPJ Executor
*EasyVVUQ-QCGPJ* provides the `Executor` class that manages the execution of potentially computationally-intensive *EasyVVUQ* operations processed on samples, in particular the encoding and model execution. The first step is to construct the `Executor` instance.

In [ ]:
qcgpjexec = Executor(my_campaign)

#### Initialisation of QCG-PilotJob Manager
Now the executor needs to be connected with *QCG-PilotJob Manager* instance. The easiest way is to use a dedicated method `create_manager`. In case of the execution of the workflow inside a SLURM computing job, the method could be invoked even without arguments. Here, since we are not in a regular allocation on HPC machine, we have to use *QCG-PilotJob Manager* in so called *local mode*. It means that we have to manually instruct the service how many resources (possibly virtual ones) we want to use. 

In [ ]:
qcgpjexec.create_manager(resources='4')

#### EasyVVUQ-QCGPJ Tasks

The next step is the setup of tasks skeletons, that will be used to generate specific tasks for the execution of demanding *EasyVVUQ* operations inside the *QCG-PilotJob* manager. To this end the `Executor` class provides a method called `add_task` that takes `Task` as an argument. In order to create a `Task` you need to provide its type, requirements and possibly Task type specific parameters. In our example we will define two tasks: `TaskType.ENCODING` for the input parameters encoding and `TaskType.EXECUTION` for the model execution. 

##### *Encoding task*
With the following task definition we specify that each encoding of a sample should be executed on a single core:

In [ ]:
qcgpjexec.add_task(Task(
    TaskType.ENCODING,
    TaskRequirements(cores=1)
))

##### *Model execution task*
The task definition for a model execution is a bit more complicated. Along with the type, and requirements, we need to specify the application that will be executed for each sample. Thus the constructor for this task type takes one additional named parameter, called `application`. In this example we specify that our application is a python model that takes the encoded file (`cooling_model.py`) as an input. As you can see, for the execution of tasks of this kind we request for 4 cores.  

In [ ]:
qcgpjexec.add_task(Task(
    TaskType.EXECUTION,
    TaskRequirements(cores=4),
    application='python3 ' + os.path.abspath("EasyVVUQ-QCGPJ/tests/cooling/cooling_model.py") + " cooling_in.json"
))

<div style="color: gray; border: dotted 1px gray; padding: 8px">
    <b>Note:</b> there is also a third type of Task in EasyVVUQ-QCGPJ, namely TaskType.EXECUTION_ENCODING. This type of a task can be defined to enable integrated run of encoding and model execution for a sample inside a single QCG-PilotJob task.
</div>

#### Run encodings and executions with EasyVVUQ-QCGPJ 
Once all the above steps are completed, we can use *Executor's* `run` method to encode and execute all samples stored in a `Campaing` using *QCG-PilotJob*. The first parameter of the `run` method is the `campaing` object itself. The method gives also the possibility to select a `submit_order`'s. Here we select the `SubmitOrder.RUN_ORIENTED` scheme, which means that the tasks are submitted in a priority of RUN (sample); in other words we want to complete whole processing (encoding and execution) for a given sample as soon as possible and then go to the next sample. 

In [ ]:
qcgpjexec.run(submit_order=SubmitOrder.RUN_ORIENTED)

<div style="color: gray; border: dotted 1px gray; padding: 8px">
    <b>Note:</b> EasyVVUQ-QCGPJ provides a few SubmitOrder schemes, namely: RUN_ORIENTED, PHASE_ORIENTED, RUN_ORIENTED_CONDENSED or EXECUTION_ONLY. Depending on your use-case it may be more beneficial to select one scheme than the other. You can find description of the SubmitOrders at the http://easyvvuq-qcgpj.readthedocs.io. 
</div>

#### Terminate QCG-PilotJob Manager
When you completed the execution, the QCG-PilotJob Manager service still waits on possible further connections. The good practice is to terminate it.

In [ ]:
qcgpjexec.terminate_manager()

At this place the code specific for *EasyVVUQ-QCGPJ* ends. We can continue with the pure *EasyVVUQ* code.

### Collating data and analysis
Now we can collate generated results and then make analysis in a typical for *EasyVVUQ* way. The code below is simply a copy from the example distributed with the *EasyVVUQ*. Therefore, if you want to get more information about the logic behind these code please go to the basic *EasyVVUQ* tutorial.  

In [ ]:
my_campaign.collate()

my_analysis = uq.analysis.PCEAnalysis(sampler=my_sampler, qoi_cols=["te"])
my_campaign.apply_analysis(my_analysis)

results = my_campaign.get_last_analysis()

mean = results.describe()['te']['mean']
std = results.describe()['te']['std']
p10 = results.describe()['te']['10%']
p90 = results.describe()['te']['90%']

s1_kappa = results._get_sobols_first('te', 'kappa')
s1_t_env = results._get_sobols_first('te', 't_env')

t = np.linspace(0, 200, 150)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16,6))

ax1.plot(t, mean, 'g-', alpha=0.75, label='Mean')
ax1.plot(t,  p10, 'b-', alpha=0.25)
ax1.plot(t,  p90, 'b-', alpha=0.25)
ax1.fill_between(
  t,
  p10,
  p90,
  alpha=0.25,
  label='90% prediction interval')
ax1.set_xlabel('Time (min)')
ax1.set_ylabel('Temperature (°C)', color='b')
ax1.tick_params('y', colors='b')
ax1.legend()

ax1t = ax1.twinx()
ax1t.plot(t, std, 'r-', alpha=0.5)
ax1t.set_ylabel('Standard deviation', color='r')
ax1t.tick_params('y', colors='r')

ax1.grid()
ax1.set_title('Statistical Moments')

ax2.plot(t, s1_kappa, '-', color ='#248BF2', label=r'$\kappa$')
ax2.plot(t, s1_t_env, '-', color ='#9402E8', alpha=0.6,label=r'$T_{env}$')

ax2.set_xlabel('Time (min)')
ax2.set_ylabel('First-order Sobol indices')
ax2.set_title('Sensitivity Analysis')
ax2.grid()
ax2.legend()

plt.subplots_adjust(wspace=0.35)

plt.show()

## Clean

### Clean outputs

In [ ]:
!rm -rf coffee_pce*
!rm -rf .qcg*
print("Done")

### Clean deployment

In [ ]:
!rm -rf EasyVVUQ-QCGPJ
print("Done")